In [1]:
import glob
import os
import sys
import pandas as pd

import nltk
#nltk.download()   # comment after first download
from nltk.tokenize import MWETokenizer
from nltk.corpus import stopwords
from FrenchLefffLemmatizer.FrenchLefffLemmatizer import FrenchLefffLemmatizer
import string
from numbers import Number
from pprint import pprint
import logging
import operator

pd.options.display.max_rows = 30

In [2]:
fr_file = './Accounts/q3q4_output/*.csv'

keywords_chosen = '13_FSDS_Goals_Keywords_FR.csv'

OUTPUT_COLS = ['id', 'num_comments', 'num_shares',
               'num_angrys', 'num_hahas', 'num_likes', 'num_loves', 'num_sads',
                'num_special', 'num_wows', 'Reactions_SUM', 
               'FSDS_matched_keywords', 'FSDS_category']

In [3]:
# read csv files and save targt columns to dataframe
# all files in q3q4output get merged into one df
filePaths = glob.glob(fr_file)
data_df = pd.DataFrame(columns=OUTPUT_COLS)
for filename in filePaths:
    #print(filename)    
    df_i = pd.read_csv(filename, encoding = 'utf-8')
    data_df = data_df.append(df_i, ignore_index=True)

data_df = data_df[OUTPUT_COLS]
data_df

,id,num_comments,num_shares,num_angrys,num_hahas,num_likes,num_loves,num_sads,num_special,num_wows,Reactions_SUM,FSDS_matched_keywords,FSDS_category
0,928380207253385217,NaN,111,NaN,NaN,0,NaN,NaN,NaN,NaN,111,[],unknown
1,935625178683461634,NaN,93,NaN,NaN,0,NaN,NaN,NaN,NaN,93,[],unknown
2,884416668327346178,NaN,85,NaN,NaN,0,NaN,NaN,NaN,NaN,85,"['emploi', 'débouché', 'entreprise', 'entrepri...",Clean growth
3,928712688552808448,NaN,83,NaN,NaN,0,NaN,NaN,NaN,NaN,83,[],unknown
4,914956862445031430,NaN,45,NaN,NaN,0,NaN,NaN,NaN,NaN,45,[],unknown
5,912780595356422145,NaN,45,NaN,NaN,0,NaN,NaN,NaN,NaN,45,['scientifique'],Clean growth
6,913778721626181632,NaN,45,NaN,NaN,0,NaN,NaN,NaN,NaN,45,"['communauté', 'communauté']",Connecting Canadians with nature
7,882342304308232193,NaN,37,NaN,NaN,0,NaN,NaN,NaN,NaN,37,"['commerce', 'transport']",Clean growth
8,839211264580603904,NaN,10,NaN,NaN,25,NaN,NaN,NaN,NaN,35,[],unknown
9,871772078986362880,NaN,34,NaN,NaN,0,NaN,NaN,NaN,NaN,34,"['communauté', 'communauté']",Connecting Canadians with nature


In [4]:
# set of punctuations to remove from text
exclude = set(string.punctuation)

stopWords = set(stopwords.words('french'))

lemma = FrenchLefffLemmatizer()    # French lemmatizer


# lemmatize_keywords also clears 'nan' from input keyword list file
# lemmatization is conducted based on context, some words may not get lemmatized, 
# e.g. "local eating" does not get lemmatized to "local eat"
def lemmatize_keywords(col):
    if str(col).lower() == 'nan':
        return ''
    return '_'.join(lemma.lemmatize(word).lower() for word in col.replace('’', '\'').replace('.', '').split()) #


In [5]:
# load keywords list
pd.options.display.max_rows = 100
keywords_df = pd.read_csv(keywords_chosen, encoding='utf-8')   # "ISO-8859-1"
KEYWORDS_COLS = keywords_df.columns
lemma_keywords_df = pd.DataFrame(columns=KEYWORDS_COLS)
category_dict = {}
keywords_list = set()
for col in KEYWORDS_COLS:
    lemma_keywords_df[col] = keywords_df[col].astype(str).apply(lemmatize_keywords)
    category_dict[col.lower()] = set(lemma_keywords_df[col].tolist())
    #category_dict[col.lower()].remove('')
    keywords_list = keywords_list.union(category_dict[col.lower()])


In [6]:
data_df['FSDS_category'] = data_df['FSDS_category'].str.lower()

In [7]:
data_df['FSDS_category'] = data_df['FSDS_category'].str.replace('unknown', 'other')
#data_df['FSDS_category'] = data_df['FSDS_category'].str.replace('connecting canadians with nature ', 'connecting canadians with nature')

In [8]:
output_df = data_df.groupby(['FSDS_category']).sum().reset_index()
output_df#.columns = ['FSDS_category', 'sentiment', 'senti_count']

,FSDS_category,num_comments,num_shares,num_angrys,num_hahas,num_likes,num_loves,num_sads,num_special,num_wows,Reactions_SUM,FSDS_matched_keywords
0,clean drinking water,16,1520,4,0,792,18,0,0,0,2350,"['santé', 'santé']['santé', 'santé']['santé', ..."
1,clean energy,44,841,2,8,813,22,1,0,2,1733,['pétrole_brut']['pétrole_brut']['changement_c...
2,clean growth,989,17508,28,53,11569,443,4,13,72,30679,"['emploi', 'débouché', 'entreprise', 'entrepri..."
3,connecting canadians with nature,1566,11525,2,49,41229,1590,63,12,365,56401,"['communauté', 'communauté']['communauté', 'co..."
4,effective action on climate change,58,4591,4,2,2120,31,0,1,6,6813,"['#changementsclimatiques', '#changementsclima..."
5,healthy coasts and oceans,365,17482,10,9,14313,350,12,8,69,32618,"['conservation', 'conservation', 'conservation..."
6,healthy wildlife populations,309,3454,3,6,12115,285,19,3,52,16246,"['air', 'air', 'santé', 'santé']['#faune']['oi..."
7,low-carbon government,220,2924,1,16,4681,216,1,3,32,8094,"['entreprise', 'entreprise', 'entreprise']['en..."
8,modern and resilient infrastructure,59,1432,2,1,1030,34,2,0,12,2572,"['producteur', 'producteur', 'canola', 'canola..."
9,other,3107,73997,118,259,37534,1039,45,29,291,116419,[][][][][][][][][][][][][][][][][][][][][][][]...


In [9]:
def find_topics(row):
    matched_words = row['FSDS_matched_keywords'].replace('\', \'', ',').replace('\'][\'', ',').replace('[\'', '').replace('\']', '').replace('[]', '')
    if matched_words == '':
        return ''
    print('Processing FSDS Goal: ' + row['FSDS_category'])
    matched_word_list = matched_words.split(',')
    matched_word_set = set(matched_word_list)
    remove_words = []
    for word in matched_word_set:
        if word not in category_dict[row['FSDS_category']]:
            remove_words.append(word)
            #print('This word does not belong to this category: ' + word)
    for word in remove_words:
        matched_word_set.remove(word)
    counter = {}
    for word in matched_word_set:
        counter[word] = 0
    #print(counter)
    for word in matched_word_list:
        if word in category_dict[row['FSDS_category']]:
            counter[word] += 1
    print(counter)
    result = sorted(counter, key=counter.get, reverse=True)
    
    return str(', '.join(result[0:8]))

In [10]:
output_df.columns
output_df['Topics/Issues'] = output_df.apply(find_topics, axis=1)

Processing FSDS Goal: clean drinking water
{'salubrité': 6, 'traitement': 28, 'technologie': 42, 'prospér': 2, 'autochtone': 102, 'santé': 294, 'municipalité': 56, 'eau_potable': 4, 'remise': 3, 'infrastructure': 36, 'inuit': 99, '#qualitéeau': 10, 'collectivité': 334, 'potable': 22}
Processing FSDS Goal: clean energy
{'pétrole': 4, 'hydrocarbure': 4, 'croissance_économique': 8, 'dioxyde_de_carbone': 4, 'innovation': 3, 'carbone': 45, 'électricité': 12, 'nucléaire': 13, 'énerguide': 3, '#génénergie': 273, 'renouvelable': 9, '#technologiespropres': 33, 'bioénergie': 4, 'énergie_propre': 3, 'fiable': 18, 'efficacité_énergétique': 6, '#techpropre': 15, 'changement_climatique': 30, '#changementclimatique': 8, 'solaire': 42, '#canwea': 3, 'gaz_naturel': 7, 'consommation_fantôme': 1, 'polluant': 128, 'pétrole_brut': 8, 'biomasse': 2, 'éolienne': 21, 'méthane': 12}
Processing FSDS Goal: clean growth
{'émission': 312, '#entreprisealimentaire': 37, 'revenu': 131, 'export': 4, 'croissance_économ

In [11]:
REACTION_COLS = ['num_comments', 'num_shares', 'num_angrys', 'num_hahas', 'num_likes', 'num_loves', 'num_sads',
                'num_special', 'num_wows',]
def merge_reaction(row):
    merge = ''
    for col in REACTION_COLS:
        merge += col[4:] + ': ' + str(row[col]) + '\n'
    merge += 'Reactions SUM: ' + str(row["Reactions_SUM"])
    return merge
    

In [12]:
output_df['Reaction summary'] = output_df.apply(merge_reaction, axis=1)
output_df = output_df[['FSDS_category', 'Reaction summary', 'Topics/Issues']]
output_df.columns = ['FSDS Goal', 'Reaction Summary', 'Topics/Issues']
output_df['FSDS Goal'] = output_df['FSDS Goal'].str.title()
output_df

C:\Users\brieh\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,FSDS Goal,Reaction Summary,Topics/Issues
0,Clean Drinking Water,comments: 16\nshares: 1520\nangrys: 4\nhahas: ...,"collectivité, santé, autochtone, inuit, munici..."
1,Clean Energy,comments: 44\nshares: 841\nangrys: 2\nhahas: 8...,"#génénergie, polluant, carbone, solaire, #tech..."
2,Clean Growth,comments: 989\nshares: 17508\nangrys: 28\nhaha...,"produit, scientifique, travail, protéger, sect..."
3,Connecting Canadians With Nature,comments: 1566\nshares: 11525\nangrys: 2\nhaha...,"parc, parc_national, activité, nature, visite,..."
4,Effective Action On Climate Change,comments: 58\nshares: 4591\nangrys: 4\nhahas: ...,"technologie, #changementsclimatiques, pollutio..."
5,Healthy Coasts And Oceans,comments: 365\nshares: 17482\nangrys: 10\nhaha...,"pêche, autochtone, conservation, poisson, saum..."
6,Healthy Wildlife Populations,comments: 309\nshares: 3454\nangrys: 3\nhahas:...,"conservation, protection, parc_national, oisea..."
7,Low-Carbon Government,comments: 220\nshares: 2924\nangrys: 1\nhahas:...,"entreprise, carbone, émission, fédéral, constr..."
8,Modern And Resilient Infrastructure,comments: 59\nshares: 1432\nangrys: 2\nhahas: ...,"transport, #listedesurveillance, déraillement,..."
9,Other,comments: 3107\nshares: 73997\nangrys: 118\nha...,


In [15]:
output_df['Reaction Summary']

0     comments: 16\nshares: 1520\nangrys: 4\nhahas: ...
1     comments: 44\nshares: 841\nangrys: 2\nhahas: 8...
2     comments: 989\nshares: 17508\nangrys: 28\nhaha...
3     comments: 1566\nshares: 11525\nangrys: 2\nhaha...
4     comments: 58\nshares: 4591\nangrys: 4\nhahas: ...
5     comments: 365\nshares: 17482\nangrys: 10\nhaha...
6     comments: 309\nshares: 3454\nangrys: 3\nhahas:...
7     comments: 220\nshares: 2924\nangrys: 1\nhahas:...
8     comments: 59\nshares: 1432\nangrys: 2\nhahas: ...
9     comments: 3107\nshares: 73997\nangrys: 118\nha...
10    comments: 136\nshares: 2857\nangrys: 4\nhahas:...
11    comments: 119\nshares: 2099\nangrys: 0\nhahas:...
12    comments: 294\nshares: 2178\nangrys: 14\nhahas...
13    comments: 3591\nshares: 3090\nangrys: 0\nhahas...
Name: Reaction Summary, dtype: object

In [14]:
output_df.to_csv('./section3-3_table_fr.csv', index=None)